In [90]:
import pandas as pd
import re

In [91]:
novel = pd.read_csv('novels_clean')

In [92]:
novels = novel.drop('Unnamed: 0',axis=1)
novels

,Title,Genre,Rating,Author,Published Year,Title id
0,Trash of the Count’s Family,"Action , Adventure , Comedy , Drama , Fantasy ...","(4.5 / 5.0, 2965 votes)",Yu Ryeo Han,2018,953
1,Everyone Else is a Returnee,"Action , Adventure , Comedy , Fantasy , Harem ...","(4.3 / 5.0, 3458 votes)",Toika,2016,183
2,The Death Mage Who Doesn’t Want a Fourth Time,"Action , Adventure , Comedy , Fantasy , Harem ...","(4.2 / 5.0, 2260 votes)",Densuke,2015,765
3,Tsuki ga Michibiku Isekai Douchuu,"Action , Adventure , Comedy , Drama , Fantasy ...","(3.8 / 5.0, 1893 votes)",Azumi Kei,2013,957
4,"Kumo Desu ga, Nani ka?","Action , Adventure , Comedy , Drama , Fantasy ...","(4.5 / 5.0, 1837 votes)",Baba Okina,2015,422
...,...,...,...,...,...,...
1020,The Skill Maker,"Action , Adventure , Drama , Fantasy , Shounen...","(3.7 / 5.0, 234 votes)",Dong Jeon Paseu,2016,869
1021,The General’s Little Peasant Wife,"Adventure , Drama , Josei , Romance , Supernat...","(3.5 / 5.0, 225 votes)",陽光小葉,2015,787
1022,The Kind Older Sister Is No More,"Drama , Psychological , Romance ,","(4.1 / 5.0, 150 votes)",Yun Yeo-eum,2018,804
1023,Avoid the Death Route!,"Action , Comedy , Fantasy , Romance , Shoujo ,...","(4.3 / 5.0, 236 votes)",Minazuki Shizuru,2017,64


In [93]:
def clean_titles(Title):
    return re.sub("[^a-zA-Z0-9 ]","",Title)

In [94]:
novels["clean_titles"] = novels["Title"].apply(clean_titles)

In [95]:
new_column_name = 'title_id'
novels.rename(columns={'Title id': new_column_name}, inplace=True)

In [96]:
novels

,Title,Genre,Rating,Author,Published Year,title_id,clean_titles
0,Trash of the Count’s Family,"Action , Adventure , Comedy , Drama , Fantasy ...","(4.5 / 5.0, 2965 votes)",Yu Ryeo Han,2018,953,Trash of the Counts Family
1,Everyone Else is a Returnee,"Action , Adventure , Comedy , Fantasy , Harem ...","(4.3 / 5.0, 3458 votes)",Toika,2016,183,Everyone Else is a Returnee
2,The Death Mage Who Doesn’t Want a Fourth Time,"Action , Adventure , Comedy , Fantasy , Harem ...","(4.2 / 5.0, 2260 votes)",Densuke,2015,765,The Death Mage Who Doesnt Want a Fourth Time
3,Tsuki ga Michibiku Isekai Douchuu,"Action , Adventure , Comedy , Drama , Fantasy ...","(3.8 / 5.0, 1893 votes)",Azumi Kei,2013,957,Tsuki ga Michibiku Isekai Douchuu
4,"Kumo Desu ga, Nani ka?","Action , Adventure , Comedy , Drama , Fantasy ...","(4.5 / 5.0, 1837 votes)",Baba Okina,2015,422,Kumo Desu ga Nani ka
...,...,...,...,...,...,...,...
1020,The Skill Maker,"Action , Adventure , Drama , Fantasy , Shounen...","(3.7 / 5.0, 234 votes)",Dong Jeon Paseu,2016,869,The Skill Maker
1021,The General’s Little Peasant Wife,"Adventure , Drama , Josei , Romance , Supernat...","(3.5 / 5.0, 225 votes)",陽光小葉,2015,787,The Generals Little Peasant Wife
1022,The Kind Older Sister Is No More,"Drama , Psychological , Romance ,","(4.1 / 5.0, 150 votes)",Yun Yeo-eum,2018,804,The Kind Older Sister Is No More
1023,Avoid the Death Route!,"Action , Comedy , Fantasy , Romance , Shoujo ,...","(4.3 / 5.0, 236 votes)",Minazuki Shizuru,2017,64,Avoid the Death Route


In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(ngram_range=(1,2))
tfidf = vec.fit_transform(novels["clean_titles"])

In [98]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search (title):
    title = clean_titles(title)
    q_vec = vec.transform([title])
    similarity = cosine_similarity(q_vec,tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = novels.iloc[indices][::-1]
    return results

In [99]:
from ipywidgets import widgets
from IPython.display import display

novels_input = widgets.Text(
    value = "Search Here",
    description = "Novels Title:",
    disabled = False
)
novel_list = widgets.Output()

def on_type(data):
    with novel_list:
        novel_list.clear_output()
        title = data["new"]
        if len(title) > 3:
            display(search(title))

novels_input.observe(on_type, names = 'value')
display(novels_input,novel_list)

Text(value='Search Here', description='Novels Title:')

Output()

In [100]:
rating = pd.read_csv('novel_rating_clean')
ratings = rating.drop("Unnamed: 0",axis=1)
new_column_name = 'title_id'
ratings.rename(columns={'Title id': new_column_name}, inplace=True)
new_column_name = 'user_id'
ratings.rename(columns={'User id': new_column_name}, inplace=True)
new_column_name = 'title'
ratings.rename(columns={'Title': new_column_name}, inplace=True)

In [101]:
ratings

,title,User Name,Rating,title_id,user_id
0,Trash of the Count’s Family,Ztarfish,0,953,18786
1,Trash of the Count’s Family,Len_,5,953,9202
2,Trash of the Count’s Family,Nocta,5,953,11887
3,Trash of the Count’s Family,Jass,2,953,7525
4,Trash of the Count’s Family,sesamechikn,5,953,27124
...,...,...,...,...,...
93519,Jiang Hu’s Road is Curved,yourstrulylen,0,400,29289
93520,Jiang Hu’s Road is Curved,miaomiaomiao28,0,400,24807
93521,Jiang Hu’s Road is Curved,Kubera,5,400,8720
93522,Jiang Hu’s Road is Curved,lourdes,5,400,24312


In [102]:
titleide = 788

In [103]:
similar_user = ratings[(ratings["title_id"] ==titleide) & (ratings["Rating"] > 3)]['user_id'].unique()

In [104]:
similar_user_rec = ratings[(ratings["user_id"].isin(similar_user)) & (ratings["Rating"] > 3)]["title_id"]
similar_user_rec = similar_user_rec.value_counts() / len(similar_user)
similar_user_rec = similar_user_rec[similar_user_rec>.1]
similar_user_rec

title_id
788    1.0
Name: count, dtype: float64

In [105]:
all_user = ratings[(ratings["title_id"].isin(similar_user_rec.index)) & (ratings["Rating"] > 3)]
all_user

,title,User Name,Rating,title_id,user_id
43828,The Girl Who Ate a Death God,JerryHatrick,5,788,7613
43830,The Girl Who Ate a Death God,SODTranslations,5,788,14302
43831,The Girl Who Ate a Death God,Mizura,5,788,10932
43832,The Girl Who Ate a Death God,Simple As It,5,788,15245
43833,The Girl Who Ate a Death God,viper764,5,788,28645
...,...,...,...,...,...
43909,The Girl Who Ate a Death God,BanetheCatman,5,788,1977
43910,The Girl Who Ate a Death God,Demonic Reader lv 451F,5,788,4046
43911,The Girl Who Ate a Death God,Bivo Alexandre,5,788,2207
43912,The Girl Who Ate a Death God,MangoGuy,4,788,10236


In [106]:
all_user_rec = all_user["title_id"].value_counts() / len(all_user['user_id'].unique())
all_user_rec

title_id
788    1.0
Name: count, dtype: float64

In [107]:
rec_percentages = pd.concat([similar_user_rec,all_user_rec], axis=1)
rec_percentages.columns = ['similar','all']
rec_percentages['scores'] = rec_percentages['similar'] / rec_percentages['all']
rec_percentages.sort_values('scores', ascending=False)

,similar,all,scores
title_id,,,
788,1.0,1.0,1.0


In [108]:
rec_percentages.head(5).merge(novels, left_index=True, right_on="title_id")

,similar,all,scores,Title,Genre,Rating,Author,Published Year,title_id,clean_titles
294,1.0,1.0,1.0,The Girl Who Ate a Death God,"Action , Fantasy , Mature , Psychological , Se...","(4.8 / 5.0, 782 votes)",Nanasawa Matari,2012,788,The Girl Who Ate a Death God


In [109]:
def find_similar_novels(title_id):
    similar_user = ratings[(ratings["title_id"] ==title_id) & (ratings["Rating"] > 3)]['user_id'].unique()
    similar_user_rec = ratings[(ratings["user_id"].isin(similar_user)) & (ratings["Rating"] > 3)]["title_id"]
    
    similar_user_rec = similar_user_rec.value_counts() / len(similar_user)
    similar_user_rec = similar_user_rec[similar_user_rec>0.10]
    
    all_user = ratings[(ratings["title_id"].isin(similar_user_rec.index)) & (ratings["Rating"] > 3)]
    all_user_rec = all_user["title_id"].value_counts() / len(all_user['user_id'].unique())
    
    rec_percentages = pd.concat([similar_user_rec,all_user_rec], axis=1)
    rec_percentages.columns = ['similar','all']
    
    rec_percentages['scores'] = rec_percentages['similar'] / rec_percentages['all']
    
    rec_percentages.sort_values('scores', ascending=False)
    
    return rec_percentages.head(10).merge(novels, left_index=True, right_on="title_id")[['scores','Title','Genre','Rating','Author','Published Year']]

In [110]:
novels_input = widgets.Text(
    value = "Search Here",
    description = "Novels Title:",
    disabled = False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 3:
            results = search(title)
            title_id = results.iloc[0]['title_id']
            display(find_similar_novels(title_id))

novels_input.observe(on_type, names = 'value')
display(novels_input, recommendation_list)

Text(value='Search Here', description='Novels Title:')

Output()